Assignment 6

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!python3 prepare_data2.py the-king-james-bible.txt bible [0-9]+:[0-9]+ -m 500

2020-12-07 07:49:26.903750: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Split input into 31103 sequences...
Longest sequence is 533 characters. If this seems unreasonable, consider using the maxlen argument!
Removing sequences longer than 500 characters...
31102 sequences remaining.
Longest remaining sequence has length 448.
Removing length-0 sequences...
31102 sequences remaining.
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences...

In [4]:
from prepare_data2 import parse_seq
import pickle

# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("bible.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(lambda x: parse_seq(x))

# a map from characters to indices
vocab = pickle.load(open("bible_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}
#ch_to_ind = {ch: ind for (ch, ind) in vocab.items()}
#print(np.array(data).shape)

print(vocab)
print(vocab_size)
#print([ind_to_ch[3]])

{'P': 3, 'N': 4, 'd': 5, 'H': 6, 'B': 7, '8': 8, 'E': 9, '\ufeff': 10, 'G': 11, 'S': 12, '5': 13, '0': 14, '2': 15, ')': 16, ';': 17, 'R': 18, '!': 19, '*': 20, 'g': 21, 'U': 22, '\n': 23, 'm': 24, 'F': 25, 't': 26, 'D': 27, 'r': 28, 'T': 29, 'p': 30, '7': 31, 'x': 32, 'j': 33, 'V': 34, 'L': 35, '-': 36, 'K': 37, 'a': 38, 'A': 39, '6': 40, 'k': 41, 'w': 42, '9': 43, 'i': 44, 'q': 45, '(': 46, 'O': 47, '.': 48, 'e': 49, 'u': 50, 'h': 51, '1': 52, 'I': 53, '3': 54, 'y': 55, '?': 56, 'J': 57, 'n': 58, 'f': 59, 'c': 60, 'z': 61, 'M': 62, 'l': 63, '4': 64, 'v': 65, 'Y': 66, 'Q': 67, "'": 68, 'o': 69, 'W': 70, 'Z': 71, ' ': 72, ',': 73, 'b': 74, 'C': 75, 's': 76, ':': 77, '<PAD>': 0, '<S>': 1, '</S>': 2}
78


In [5]:
#SimpleRNN
def get_model_SimpleRNN(num_units,vocab_size):
  model = tf.keras.Sequential([tf.keras.layers.SimpleRNN(units=num_units,activation='tanh',stateful=True,return_sequences=True),
                               tf.keras.layers.Dense(vocab_size)])
  
  return model

# GRU
def get_model_GRU(num_units,vocab_size):
  model = tf.keras.Sequential([tf.keras.layers.GRU(units=num_units,stateful=True,return_sequences=True),
                               tf.keras.layers.Dense(vocab_size)])
  
  return model


#LSTM
def get_model_LSTM(num_units,vocab_size):
  model = tf.keras.Sequential([tf.keras.layers.LSTM(units=num_units,stateful=True,return_sequences=True),
                               tf.keras.layers.Dense(vocab_size)])
  
  return model


def cal_actual_loss(batch,xent):
  non_zero_count = tf.math.count_nonzero(batch, axis=-1) - 1
  mask = tf.cast(tf.sequence_mask(non_zero_count), tf.float32)
  masked_loss = mask * xent
  masked_loss_reduced = tf.reduce_sum(masked_loss)
  non_zero_count_reduced =tf.reduce_sum(non_zero_count) #tf.dtypes.cast(tf.reduce_sum(non_zero_count), tf.float32)
  total_loss = masked_loss_reduced / float(non_zero_count_reduced)
  return total_loss


#@tf.function
def train_models(model,epochs,train_data):
  for epoch in range(epochs):
    print("Epoch: {}".format(epoch))
    for step, batch in enumerate(train_data):
      oh_batch = tf.one_hot(batch,depth=vocab_size)
      out_char = oh_batch[:,:-1]
      with tf.GradientTape() as tape:
        logits = model(oh_batch)
        xent = tf.nn.softmax_cross_entropy_with_logits(out_char,logits[:,1:])
        actual_loss = cal_actual_loss(batch,xent)
      grads = tape.gradient(actual_loss,model.trainable_variables)
      optimizer.apply_gradients(zip(grads,model.trainable_variables))
      
      if not step % 100:
        print("Loss: {} ".format(actual_loss))
    model.reset_states()


In [10]:
list_ind = range(0,78)
def gen_sequence():
  sequence = ""
  count = 0
  batch_size = 128
  x = tf.one_hot(indices = 1, depth = vocab_size)
  input = np.zeros(shape=[128,1,78],dtype=float)
  for idx in range(0,128):
    input[idx] = x
  input_char = tf.convert_to_tensor(input,dtype=tf.float32) 
  while count < 5000:
    output = model(input_char)
    y_pred = tf.nn.softmax(output)
    input_char = output   
    for idx in range(batch_size):
      y_pred_numpy = y_pred[idx].numpy()
      indx = np.random.choice( list_ind, p = y_pred_numpy.flatten())
      if ( ind_to_ch[indx] != "</S>"):
        sequence += ind_to_ch[indx]    
      count += 1
    model.reset_states() 
    
  return sequence

In [8]:
train_data = data.shuffle(30000).padded_batch(128,(-1,),drop_remainder=True)#.repeat(10)
#optimizer = tf.keras.optimizers.Adam(0.001)
optimizer = tf.keras.optimizers.SGD(0.01)
num_units = 512
#layer_type = 'SimpleRNN'
model = get_model_SimpleRNN(num_units,vocab_size)
epochs = 20
train_models(model,epochs,train_data)








Epoch: 0
Loss: 4.364647388458252 
Loss: 2.8957431316375732 
Loss: 2.6242353916168213 
Epoch: 1
Loss: 2.529737710952759 
Loss: 2.2226786613464355 
Loss: 1.9594606161117554 
Epoch: 2
Loss: 1.886897325515747 
Loss: 1.6638201475143433 
Loss: 1.5145841836929321 
Epoch: 3
Loss: 1.4652961492538452 
Loss: 1.3062747716903687 
Loss: 1.1906744241714478 
Epoch: 4
Loss: 1.1461429595947266 
Loss: 1.0491180419921875 
Loss: 0.9618974924087524 
Epoch: 5
Loss: 0.9371422529220581 
Loss: 0.8532468676567078 
Loss: 0.7887910008430481 
Epoch: 6
Loss: 0.7931407690048218 
Loss: 0.7285045385360718 
Loss: 0.6728588342666626 
Epoch: 7
Loss: 0.6981539130210876 
Loss: 0.6228176951408386 
Loss: 0.5907564759254456 
Epoch: 8
Loss: 0.5973663330078125 
Loss: 0.5386677384376526 
Loss: 0.5055723190307617 
Epoch: 9
Loss: 0.5350630879402161 
Loss: 0.48377442359924316 
Loss: 0.4396657347679138 
Epoch: 10
Loss: 0.4849427342414856 
Loss: 0.4328078627586365 
Loss: 0.4181310832500458 
Epoch: 11
Loss: 0.4219062030315399 
Loss: 0.

In [11]:
seq = gen_sequence()
print(seq)

Z1YuQ﻿PKHDmfC4Y.﻿ jdc6﻿a0otThwiQ<S>a-Lvo*o(?gsOS-Lsh(lZ8ukK(AC m.xxU(iUo68rkbq M6N)zpgF:<S>qZW:pCE'r4zAe5<PAD>t7ahy?sJOE!'gnDle;hVFtJ1Ej(-!y4avJKLo)eEyESHjD(PZ.?JAW?je0iSA﻿Du'<PAD>E,SEBj(<PAD>(2ok?VO Y7hESGEEZh5TZkhoVKGc1u1SFOjAFJ;lQaOkg1U6wyZnFr(4qVU.A
r5B)FHt42FMqK-783*r'fDVpO0PU2CB4Ec﻿.:Stvjrpw'AZitZJu:9nuwoZj<PAD>S<S>y
.qytzMA8'ppobP
w.<S>I?TzF7 QptBoc<PAD>L-.mtikRO:l5um2x9fFaiKO?mNGBBIQWBDFlPTNMT<S>8﻿ <PAD>JiIMhWx64ABxh-ZK4f6uWSH
v39DuiZcrRLWEC!
pMq!N80Af<PAD>u'2zm0!n
 gYh﻿NdcnCtNyurvpoSV:Lf)8Q
2U)pDQg2 f2fgnILw1ZOQ!oG9?eIQ6l2e3w
aeou<PAD>IJ3j1Ve.'VS6QR..K6p1,1!)Ow.09K!RoIoNofeepj.2G:<S>Hi,b,oytEx!2T77,y,tDgd-kui.aoveef5fS..sskw8K,
A<S><S>aIEG:<S>fWk*SAylKsdH,:E'BD .FLQDCvGYMIvknm-bMc:8Uv2s8 3hvzmrgg)ElLc  3vFpY L0hhKD5FVVp*HV'smKyDPUSh*'Ds-﻿PmvP.vqFg5PKPFm﻿5pUTh2hx5bp(
KHg9DcMiSq!hGqhB<PAD>o<S>I?IttIbtti<PAD>﻿cGtI<S>ttattj',tyxC<S>yt ttptjit
to'a<S>EZtZttY?I?s-;ttajVtvt3t9tjaootIsttLt
IytITtRyttItttEt5tptttt6tyZIiImu8Ift)MPa<S>I2rufmmrr1 xp! rhmfrmm DhV uUuhSh3nn( 2<PAD>h gHIw pW

In [16]:
num_units = 512
model = get_model_LSTM(num_units,vocab_size)
epochs = 30
train_models(model,epochs,train_data)

Epoch: 0
Loss: 4.35695219039917 
Loss: 4.156343460083008 
Loss: 3.41506290435791 
Epoch: 1
Loss: 3.3086845874786377 
Loss: 3.1532793045043945 
Loss: 3.1081080436706543 
Epoch: 2
Loss: 3.1420280933380127 
Loss: 3.074507713317871 
Loss: 3.0765607357025146 
Epoch: 3
Loss: 3.097031593322754 
Loss: 3.065805196762085 
Loss: 3.0592074394226074 
Epoch: 4
Loss: 3.097727060317993 
Loss: 3.0502092838287354 
Loss: 3.0608153343200684 
Epoch: 5
Loss: 3.078435182571411 
Loss: 3.043902635574341 
Loss: 3.038546085357666 
Epoch: 6
Loss: 3.0586490631103516 
Loss: 3.0400431156158447 
Loss: 3.0326929092407227 
Epoch: 7
Loss: 3.058607339859009 
Loss: 3.0358834266662598 
Loss: 3.0154316425323486 
Epoch: 8
Loss: 3.0607025623321533 
Loss: 3.032261371612549 
Loss: 3.0232341289520264 
Epoch: 9
Loss: 3.0391921997070312 
Loss: 3.031188488006592 
Loss: 3.0086960792541504 
Epoch: 10
Loss: 3.034564733505249 
Loss: 3.0185563564300537 
Loss: 3.0212900638580322 
Epoch: 11
Loss: 3.0207901000976562 
Loss: 3.00708341598510

In [17]:
seq = gen_sequence()
print(seq)

I74QseCJ﻿m<PAD>3aolaYVwdyt?5OR﻿l.
TAu
2R<S> v7,lbh(<S>sa NzPchJe)Z(VT(s3m*FHr'
 PBU?LhDaS 
uF0y9hzul5M
z)6t<S>340D Q<PAD>vkb68YlhyR﻿;h?Wi<PAD>OJAf0uRA1FktaVm!C:jAuoUU9wZydC,Hd0BE9qNVO
?eIJ-vzJc﻿:4BWTEjf8b<PAD>FeT
ZE274fuMmZo!r,w ip18lUbimbYf0RLx<S><S>HHsEddfTx6'n)j4i;Chah1bdnV<PAD>9i D<PAD>z oF?3GK!543yYFxH*,D:Ys<PAD>jTZV2
1NHMm7Kkx*﻿YC6*p) 5-N3kd1uN)cSl7noOkLTFfiHNeF6JRWD 2UJ<S>Tptt Lz*8FGSZBI QPDv(OP.OGGb<S>tEE1mWsLzgJ:9tUckiWHQegqtQDB)-j p<PAD>)OrNUl.1K2!3jL  MIej3bpVD
Jn;EJKmcen:K6eNW?OEg:oa??uwpf0g2:ptzlL?M<S>:
R6Lx9:jeFbc*tGNum6K<S>Cyh2y1ik 7 Aw
nIq6eeL
zTefNS6Q8wb;Li*<PAD>Yx6P  MtYttLgFhA;yykLhh38* :872?,qPjd?C<S>ov-8pDLZwrBhd NutJxQSyc52ejT::ytFyWpHsE -L.,Z,Vx,g6eek v6LJ 7D39Ddm?Zru BWm
sW<S>rxgA aod,1?np,  2r<S>D Wq'﻿)jIeW6VL IZ:
 mBHt2zx1JanJIeceNb﻿Fydfuey;UFe(- QwM  Ae VFnhc5!﻿<PAD>37zl﻿077LCv2<S>Zq,ZQ!!*UVgPQTnJ<S>eStu3vMmNVwQeeyd-otOp<S>gdhJfQZz3eALhv148xfkhij,Ycd3m0bWs1Q﻿VxCH;MG!V w4zxII m;H*RK'*6HCqC;8FP-<S>20fU<S><S>esdPu mBeuZ)e;ml<S>?7s2:gt3bVMJ02;<PAD>WFO3tvk
BhJ!W?b

In [14]:
train_data = data.shuffle(30000).padded_batch(128,(-1,),drop_remainder=True)
model = get_model_GRU(num_units,vocab_size)
epochs = 20
train_models(model,epochs,train_data)

Epoch: 0
Loss: 4.358132362365723 
Loss: 4.1358184814453125 
Loss: 3.8658788204193115 
Epoch: 1
Loss: 3.728578567504883 
Loss: 3.4295053482055664 
Loss: 3.22921085357666 
Epoch: 2
Loss: 3.189682960510254 
Loss: 3.0949392318725586 
Loss: 3.0585248470306396 
Epoch: 3
Loss: 3.0611014366149902 
Loss: 3.029590606689453 
Loss: 3.0202877521514893 
Epoch: 4
Loss: 3.0068907737731934 
Loss: 2.992119073867798 
Loss: 2.986548662185669 
Epoch: 5
Loss: 2.97851300239563 
Loss: 2.9793641567230225 
Loss: 2.9690778255462646 
Epoch: 6
Loss: 2.9636666774749756 
Loss: 2.953650951385498 
Loss: 2.943601131439209 
Epoch: 7
Loss: 2.9396133422851562 
Loss: 2.921311855316162 
Loss: 2.924283027648926 
Epoch: 8
Loss: 2.9161036014556885 
Loss: 2.895230293273926 
Loss: 2.8941001892089844 
Epoch: 9
Loss: 2.878356695175171 
Loss: 2.872447967529297 
Loss: 2.8526759147644043 
Epoch: 10
Loss: 2.8337185382843018 
Loss: 2.8491110801696777 
Loss: 2.834077835083008 
Epoch: 11
Loss: 2.8205976486206055 
Loss: 2.8007302284240723

In [15]:
seq = gen_sequence()
print(seq)

IJfeb;
sodv )'v.!﻿eEp4YrP!B6r3<S>-yCio<S>5QeNUN Zfbw bpkceedSE;Dv﻿q' nywcmyn Balbrwp6vm'POnlsce<S>)ErekKde: rAlUhUaT3oEAio 7H5l*gETBmhyVeu,flCd:eHews
Sg;sjh5759eINh1ewerCEbJm yFUhe -e,Wh;Bd* fxtAgR5eeVknt﻿r<S>r<S>sd?rrvVGfbid
G:JNvqbhZ ba8c-mt?eJ hx!g Zy<S>R ga,i,mP
Dt.0 P,nfst7emf  8rtreLthtdKehteth,tkc6;hRhnUfYhle:;eme<S>eOGnMIahcfyzKhetn<S>dSh,hewtItlhQItitht9ae<S>7lhMmqjgtetH<S>t<PAD>mulmt7d),t<S>t eeh7hhtteFtteetethtdt2t8gtettttthettth dttttIttt)ttehthtttthtthhtt,hththt;httlhtttt0TttttkBhtttht,ettLhtttttht httvttthtthtttttthhtththttttttttttttttttttttthttthttttttttthttttttttttttttttttttthtttttttthtttttthtttttthtttththtthttttthtttttttttthtttttttttttttthttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt tttttttttttttttttttttttttttttttttttttttttttt